In [ ]:
!pip install aequitas
import yaml
import os
import pandas as pd
import numpy as np
import seaborn as sns
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
#import aequitas.plot as ap
DATAPATH = 'https://github.com/dssg/fairness_tutorial/raw/master/data/'

# Model with Highest Precision at top 1000

In [ ]:
preds = pd.read_csv(DATAPATH + 'predictions_c598fbe93f4c218ac7d325fb478598f1.csv.gz', compression='gzip')

In [ ]:
preds.shape

# Thresholding


In [ ]:
preds.head()

In [ ]:
preds = preds.sort_values('predict_proba', ascending = False).reset_index(drop=True).copy()

preds['score'] = preds.apply(lambda x: 1.0 if int(x.name)  < 1000 else 0.0, axis=1)

In [ ]:
preds['score'].value_counts()

In [ ]:
preds.rename(columns={'quickstart_label':'label_value'}, inplace=True)

In [ ]:
preds.head()

In [ ]:
attr_df = pd.read_csv(DATAPATH + 'test_20121201_20130201_protected.csv.gz', compression='gzip')
attr_df.tail(10)

In [ ]:
df = pd.merge(preds[['entity_id','as_of_date','score','label_value']], attr_df, how='left', on=['entity_id','as_of_date'], left_index=True, right_index=False, sort=True, copy=True)

In [ ]:
df.tail()

In [ ]:
df['poverty_level'].value_counts()

In [ ]:
df['metro_type'].value_counts()

In [ ]:
df['teacher_sex'].value_counts()

In [ ]:
df['label_value'].sum() / len(df)

## Aequitas

In [ ]:
g = Group()
xtab, _ = g.get_crosstabs(df[['score','label_value','poverty_level','metro_type', 'teacher_sex']].copy())

In [ ]:
absolute_metrics = g.list_absolute_metrics(xtab)

In [ ]:
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

In [ ]:
xtab[['attribute_name', 'attribute_value'] + absolute_metrics]

In [ ]:
b = Bias()
bdf = b.get_disparity_predefined_groups(xtab, original_df=df, ref_groups_dict={'poverty_level':'lower', 'metro_type':'suburban_rural', 'teacher_sex':'male'})

In [ ]:
metrics = ['tpr']

In [ ]:
ap.disparities(bdf, metrics, 'poverty_level', fairness_threshold = 1.30)

In [ ]:
ap.disparities(bdf, metrics, 'metro_type', fairness_threshold = 1.30)

In [ ]:
ap.disparities(bdf, metrics, 'teacher_sex', fairness_threshold = 1.30)